<a href="https://colab.research.google.com/github/Storm00212/QUANT/blob/main/Fourier_series_in_quant_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.fft import fft, ifft
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [2]:

class HestonFFTOptionPricer:
    """
    Option pricing using Fast Fourier Transform (FFT) for the Heston model
    Based on Carr-Madan (1999) methodology
    """

    def __init__(self, S0, r, T, kappa, theta, sigma, rho, v0):
        """
        Parameters:
        S0: Initial stock price
        r: Risk-free rate
        T: Time to maturity
        kappa: Mean reversion speed of variance
        theta: Long-term mean of variance
        sigma: Volatility of volatility (vol-of-vol)
        rho: Correlation between stock and variance processes
        v0: Initial variance
        """
        self.S0 = S0
        self.r = r
        self.T = T
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.rho = rho
        self.v0 = v0

    def heston_char_func(self, u):
        """
        Characteristic function for the Heston model
        This is the closed-form solution we get from the model
        """
        # Parameters for the characteristic function
        x0 = np.log(self.S0)

        # Pre-compute common terms
        d = np.sqrt((self.rho * self.sigma * u * 1j - self.kappa)**2 +
                    self.sigma**2 * (u * 1j + u**2))
        g = (self.kappa - self.rho * self.sigma * u * 1j - d) / \
            (self.kappa - self.rho * self.sigma * u * 1j + d)

        # Characteristic function components
        exp_part = np.exp(u * 1j * (x0 + self.r * self.T))

        term1 = (self.kappa * self.theta * self.T / self.sigma**2) * \
                (self.kappa - self.rho * self.sigma * u * 1j - d)

        term2 = (self.v0 / self.sigma**2) * \
                (self.kappa - self.rho * self.sigma * u * 1j - d) * \
                ((1 - np.exp(-d * self.T)) / (1 - g * np.exp(-d * self.T)))

        cf = exp_part * np.exp(term1 + term2) * \
             ((1 - g * np.exp(-d * self.T)) / (1 - g))**(-2 * self.kappa * self.theta / self.sigma**2)

        return cf